## Reading files

We will read the `.txt` files line by line and apply these filters:  

1. **Remove lines containing a WhatsApp encryption notice**  
   - ❌ **Before:** `dd/mm/yyyy, hh:mm - Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. Tap to learn more.`  
   - ✅ **After:** (Removed)  

2. **Remove lines with `<Media omitted>`**  
   - ❌ **Before:** `dd/mm/yyyy, hh:mm - Person: <Media omitted>`  
   - ✅ **After:** (Removed)  

3. **Remove lines containing email addresses**  
   - ❌ **Before:** `dd/mm/yyyy, hh:mm - Person: example@gmail.com`  
   - ✅ **After:** (Removed)  

4. **Remove lines containing links**  
   - ❌ **Before:** `dd/mm/yyyy, hh:mm - Person: https://www.example.com/`  
   - ✅ **After:** (Removed)  

5. **Replace `<This message was edited>` with an empty string**
   - ❌ **Before:** `dd/mm/yyyy, hh:mm - Person: hey, how are you? <This message was edited>`
   - ✅ **After:** `dd/mm/yyyy, hh:mm - Person: hey, how are you?`

6. **Remove lines with the text `You deleted this message`**  
   - ❌ **Before:** `dd/mm/yyyy, hh:mm - Person: You deleted this message`  
   - ✅ **After:** (Removed)  

7. **Remove lines with the text `null`**  
   - ❌ **Before:** `dd/mm/yyyy, hh:mm - Person: null`  
   - ✅ **After:** (Removed)  

8. **Remove lines with the text `created group`**  
   - ❌ **Before:** `dd/mm/yyyy, hh:mm - Person created group "group name"`  
   - ✅ **After:** (Removed)  

9. **Remove lines with the text `added you`**  
   - ❌ **Before:** `dd/mm/yyyy, hh:mm - Person added you`  
   - ✅ **After:** (Removed)  

10. **Replace tagging (`@person`) with an empty string**  
   - ❌ **Before:** `dd/mm/yyyy, hh:mm - Person: @person are you coming?`  
   - ✅ **After:** `dd/mm/yyyy, hh:mm - Person: are you coming?`  

In [3]:
import re
import pandas as pd


def read_whatsapp_chat(file_path: str) -> pd.DataFrame:
    # Define filtering patterns
    encryption_message = "Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. Tap to learn more."
    media_pattern = "<Media omitted>"
    email_pattern = r'[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}'
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    edited_message = "<This message was edited>"
    deleted_message = "You deleted this message"
    null_message = "null"
    created_group_message = "created group"
    added_you_to_group_message = "added you"
    tagging_pattern = r'@[\w]+'

    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    # Apply filters to remove unwanted lines
    filtered_lines = []
    for line in lines:
        if (
            encryption_message not in line and
            deleted_message not in line and
            null_message != line.split(" ")[-1] and
            media_pattern not in line and
            created_group_message not in line and
            added_you_to_group_message not in line and
            not re.search(email_pattern, line) and
            not re.search(url_pattern, line)
        ):
            line = line.replace(edited_message, "").strip()
            line = re.sub(tagging_pattern, "", line).strip()
            filtered_lines.append(line)

    # Regular expression to match both WhatsApp message formats (with and without am/pm)
    pattern = r'(\d{2}/\d{2}/\d{2,4}, \d{1,2}:\d{2}(?:\s?[ap]m)?) - (.*?): (.*?)(?=\n\d{2}/\d{2}/\d{2,4}, \d{1,2}:\d{2}(?:\s?[ap]m)? -|$)'
    content = '\n'.join(filtered_lines)
    messages = re.findall(pattern, content, re.DOTALL)

    df = pd.DataFrame(messages, columns=['timestamp', 'sender', 'message'])
    
    # Parse timestamps using format='mixed'
    timestamps = []
    for timestamp in df['timestamp']:
        try:
            timestamp = pd.to_datetime(timestamp, format='mixed', dayfirst=True)
        except Exception as e:
            print(f"Error parsing timestamp '{timestamp}': {e}")
            timestamp = pd.NaT
        timestamps.append(timestamp)
    
    df['timestamp'] = timestamps
        
    return df

The `all_chats` dictionary holds the content of each file as a dataframe with three columns: `timestamp`, `sender`, and `message`.  

In [4]:
from pathlib import Path

all_chats = {}
data_directory = Path("../data/private")
for file in data_directory.glob('*.txt'):
    file_name = file.stem
    all_chats[file_name] = read_whatsapp_chat(file)

## Text sequence

The text should be merged into a single sequence to prepare it for the next step, where the BPE algorithm will be applied and the text will be encoded.

In [ ]:
text_sequence = ""
for file_name in all_chats.keys():
    text_sequence += " ".join(all_chats[file_name]['message'].values)

len(text_sequence)

In [6]:
with open("../output/combined_text.txt", "w", encoding="utf-8") as f:
    f.write(text_sequence)